## Classification

Classification is the practice of bucketizing  (or categorizing) items based on common characteristics, and is an important principle of machine learning. For example, a robust spam email detection system should accurately catch and filter out which emails are spam, while letting ham (non-spam emails) through to the inbox.

For the examples following along in this book, the question for this chapter is, 

    "Can a machine distinguish between flower species based on images?" 

We will use two datasets where measurements of flower morphology are recorded along with the species for several specimens.

The **Iris dataset** is a classic dataset from the 1930s; it is one of the first modern
examples of statistical classification.

The dataset is a collection of morphological measurements of several Iris flowers. These measurements will enable us to distinguish multiple species of the flowers. Today, species are identified by their DNA fingerprints, but in the 1930s, DNA's role in genetics had not yet been discovered.

The following four attributes (**features**) of each plant were measured, giving the dataset four features in total:
    - sepal length
    - sepal width
    - petal length
    - petal width
 
The supervised learning, or classification problem we want to solve is, 

    "Given these examples, if we see a new flower out in the field, 
    could we make a good prediction about its species from its measurements?"

In [26]:
# Create datasets
# This code based on supporting material for the book
# Building Machine Learning Systems with Python
# by Willi Richert and Luis Pedro Coelho
# published by PACKT Publishing
#
# It is made available under the MIT License

import milksets.iris
import milksets.seeds

def save_as_tsv(fname, module):
    features, labels = module.load()
    nlabels = [module.label_names[ell] for ell in labels]
    with open(fname, 'w') as ofile:
        for f, n in zip(features, nlabels):
            ofile.write("\t".join(map(str, f)))
            ofile.write("\n")
            
#save_as_tsv('iris.tsv', milksets.iris)
#save_as_tsv('seeds.tsv', milksets.seeds)

## A Simple Classification Model
### Iris Speciation Dataset

In [27]:
# Compare Iris species

from matplotlib import pyplot as plt
import numpy as np
from sklearn.datasets import load_iris

data = load_iris()
features = data.data
feature_names = data.feature_names
target = data.target
target_names = data.target_names

# Create 6 subplots
fig, axes = plt.subplots(2, 3, figsize=(15, 7))
pairs = [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]

# Create distinct markers for each species of Iris
color_markers = [
    ('r', '>'),
    ('g', 'o'),
    ('b', 'x')
    ]

for i, (p0, p1) in enumerate(pairs):
    ax = axes.flat[i]
    
    for t in range(3):
        c, marker = color_markers[t]
        ax.scatter(features[target == t, p0], features[
            target == t, p1], marker=marker, c=c)
    ax.set_xlabel(feature_names[p0])
    ax.set_ylabel(feature_names[p1])
    ax.set_xticks([])
    ax.set_yticks([])
fig.tight_layout()
fig.savefig('figure1.png')

<img src="figure1.png" />

 - Red Triangles indicate Iris setosa
 - Blue Crosses indicate Iris Versicolor
 - Green Circles indicare Iris Virginica

We can see from the above plots that petal length separates Iris setosa from the other species by a wide margin, so we can use code to figure out a good cutoff point.

### Building a Classification Model

In [28]:
# Create an array of string using NumPy indexing
labels = target_names[target]

# Petal length is the feature as postion 2
plength = features[:, 2]

is_setosa = (labels == 'setosa')

# Build the boundary
max_setosa = plength[is_setosa].max()
min_non_setosa = plength[~is_setosa].min()
print('Maximum of setosa: {0}.'.format(max_setosa))
print('Minimum of others: {0}.'.format(min_non_setosa))

Maximum of setosa: 1.9.
Minimum of others: 3.0.


What this simple model shows is that any Iris from these three species with a petal length smaller than 2 is an Iris Setosa. If an Iris has a petal length longer than 2, it can be assumed to be one of the other two types with a high level of confidence. In this case, we used eye shot to be able to verify this divide. Next, we'll use machine learning.

    We run a loop over all possible features and thresholds to see which one results in better accuracy. 
    Accuracy is simply the fraction of examples that the model classifies correctly

In [29]:
# ~ is the boolean negation operator
features = features[~is_setosa]
labels = labels[~is_setosa]
# Build a new target variable, is_virigina
is_virginica = (labels == 'virginica')


In [30]:
print(feature_names, target_names)


['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)'] ['setosa' 'versicolor' 'virginica']


Next, we'll createa handful of funtions that can be resued to help fit our model to data, as well as predict its accuracy

In [31]:
def predict(model, features):
    
    # Apply learned model
    t, fi, reverse = model
    if reverse:
        return features[:, fi] <= t
    else:
        return features[:, fi] > t

In [32]:
def accuracy(features, labels, model):
    # Compute the accuracy of the model
    preds = predict(model, features)
    return np.mean(preds == labels)

In [33]:
def fit_model(features, labels):
    # Initialize best_acc to impossibly low value
    best_acc = -1.0

    for fi in range(features.shape[1]):
        # We are going to test all possible thresholds
        thresh = features[:,fi]
        for t in thresh:

            # Get the vector for feature `fi`
            feature_i = features[:, fi]
            # apply threshold `t`
            pred = (feature_i > t)
            acc = np.mean(pred == is_virginica)
            rev_acc = np.mean(pred == ~is_virginica)
            if rev_acc > acc:
                reverse = True
                acc = rev_acc
            else:
                reverse = False

            if acc > best_acc:
                best_acc = acc
                best_fi = fi
                best_t = t
                best_reverse = reverse
        #print('Best Accuracy: ', best_acc)
        #print('Best Feature Index: ', best_fi)
        #print('Best Threshold: ', best_t)
        #print('Best Reverse: ', best_reverse)
        return best_t, best_fi, best_reverse
    

# Fit the model with features 2 and 3 for a ~94% accuracy
model = (fit_model(features[:, 2:], labels))
print()
model = fit_model(features[train_data], is_virginica[train_data])


C:\Users\dooki\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  
C:\Users\dooki\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


At the end of the for loop, all the possible thresholds for all the possible features have been tested, and the variables best_fi, best_t, and best_reverse hold our model. This is all the information we need to be able to classify a new, unknown object, that is, to assign a class to it. 

The following code implements exactly this method:

In [34]:
def is_virginica_test(fi, t, reverse, example):
    #Apply threshold model to a new example
    test = example[fi] > t
    if reverse:
        test = not test
    return test

In [35]:
COLOUR_FIGURE = False

# Hand fixed thresholds:
t = 1.65
t2 = 1.75

# Features to use: 3 & 2
f0, f1 = 3, 2

if COLOUR_FIGURE:
    area1c = (1., .8, .8)
    area2c = (.8, .8, 1.)
else:
    area1c = (1., 1, 1)
    area2c = (.7, .7, .7)

# Plot from 90% of smallest value to 110% of largest value
# (all feature values are positive, otherwise this would not work very well)

x0 = features[:, f0].min() * .9
x1 = features[:, f0].max() * 1.1

y0 = features[:, f1].min() * .9
y1 = features[:, f1].max() * 1.1

fig,ax = plt.subplots(figsize=(10,5))
ax.fill_between([t, x1], [y0, y0], [y1, y1], color=area2c)
ax.fill_between([x0, t], [y0, y0], [y1, y1], color=area1c)
ax.plot([t, t], [y0, y1], 'k--', lw=2)
ax.plot([t2, t2], [y0, y1], 'k:', lw=2)
ax.scatter(features[is_virginica, f0],
           features[is_virginica, f1], 
           c='b', 
           marker='o', 
           s=30)
ax.scatter(features[~is_virginica, f0],
           features[~is_virginica, f1], 
           c='r', 
           marker='x', 
           s=30)
ax.set_ylim(y0, y1)
ax.set_xlim(x0, x1)
ax.set_xlabel(feature_names[f0])
ax.set_ylabel(feature_names[f1])
fig.tight_layout()
fig.savefig('figure2.png')

<img src="figure2.png" />

This machine classification is visualized above via a **decision boundary**, which tells us which features will result in one decision or another. In the above example, Iris Virginica, classified by the red cross datapoints will tend to fall in the white regions, while Iris Versicolor will tend to fall in the gray area, with a best accuracy of 94%.

It also shows (as a dashed line) an alternative threshold, which will achieve exactly the same accuracy. However, we want to be able to use this model to cateogrize new instances, so we're going to need to break the dataset into training and testing data.

We'll use this next example to bring in the iris data one more time and clean it up, this time removing the Setosa data since it's a low hanging fruit.

In [36]:
data = load_iris()
features = data['data'] # doing this a little differently 
labels = data['target_names'][data['target']]

is_setosa = (labels == 'setosa')
features = features[~is_setosa]
labels = labels[~is_setosa]
features = features[:, 2:]  # Limit features


# Virginica vs Non-Virginica
is_virginica = (labels == 'virginica')

# Split data into training and testing sets
test_data = np.tile([True, False], 50)
train_data = ~test_data

model = fit_model(features[train_data], is_virginica[train_data])
train_acc = accuracy(features[train_data], is_virginica[train_data], model)
test_acc = accuracy(features[test_data], is_virginica[test_data], model)

print('''\
Training Accuracy was {0:.1%}.
Testing Accuracy was {1:.1%} (N = {2}).
'''.format(train_acc, test_acc, test_data.sum()))

Training Accuracy was 90.0%.
Testing Accuracy was 80.0% (N = 50).



C:\Users\dooki\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  
C:\Users\dooki\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


Ideally, we would like to use all of the data for training and all of the data for testing as well, which is impossible. We can achieve a good approximation of this impossible ideal by a method called cross-validation. One simple form of cross-validation is leave-one-out cross-validation. We will take an example out of the training data, learn a model without this example, and then test whether the model classifies this example correctly.

In [37]:
correct = 0.0
for ei in range(len(features)):
    # select all but the one at position `ei`:
    train_data = np.ones(len(features), bool)
    train_data[ei] = False
    test_data = ~train_data
    model = fit_model(features[train_data], is_virginica[train_data])
    predictions = predict(model, features[test_data])
    correct += np.sum(predictions == is_virginica[test_data])
acc = correct/float(len(features))
print('Accuracy: {0:.1%}'.format(acc))

C:\Users\dooki\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  
C:\Users\dooki\Anaconda3\envs\py3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


Accuracy: 92.0%


The accuracy on the training data, the training accuracy, is almost always an overly optimistic estimate of how well your algorithm is doing. We should always measure and report the testing accuracy, which is the accuracy on a collection of examples that were not used for training. We can get most of the benefits of leave-one-out at a fraction of the cost by using x-fold cross-validation, where x stands for a small number


### Folds

You can use any number of folds you wish. There is a trade-off between computational efficiency (the more folds, the more computation is necessary) and accurate results (the more folds, the closer you are to using the whole of the data for training). Five folds is often a good compromise. This corresponds to training with 80 percent of your data, which should already be close to what you will get from using all the data.

When generating the folds, you need to be careful to keep them balanced. For example, if all of the examples in one fold come from the same class, then the results will not be representative.

### Building Blocks of a Classification Model

The following components are the major pieces of a any standard data classification model:

 - **The structure of the model**
  - How will it make decisions?
 - **The search procedure**
  - How do we find the model we need to use, and how do we optimize it?
 - **The gain or loss function**
  - How do we decide which of the tested options should be returned as a solution?
  
## A More Complex Model
### Seeds Dataset

This dataset consists of
measurements of wheat seeds. There are seven features that are present, which
are as follows:
 - area A
 - perimeter P
 - compactness C = 4πA/P²
  - compactness feature is not actually a new measurement, but a function of the previous two features, area and perimeter.
  - This is known as **feature engineering**, and can have a big impact on performance
  - This feature will have the same value for two kernels, one of which is twice as big as the other one, but with the same shape
  - does not vary with size, but varies with the shape
 - length of kernel
 -  width of kernel
 - asymmetry coefficient
 - length of kernel groove
 
There are three classes, corresponding to three wheat varieties: Canadian, Koma, and Rosa.

For use with this dataset, we will introduce a new classifier: the nearest neighbor classifier. The nearest neighbor classifier is very simple. When classifying a new element, it looks at the training data for the object that is closest to it. The nearest neighbor method can be generalized to look not at a single neighbor, but to multiple ones and take a vote amongst the neighbors. This makes the method more robust to outliers or mislabeled data.

We will use sciki-learn and it's prebuilt robust libraries to help with exploring this task.

In [38]:
def load_dataset(dataset_name):
    '''
    data,labels = load_dataset(dataset_name)

    Load a given dataset

    Returns
    -------
    data : numpy ndarray
    labels : list of str
    '''
    data = []
    labels = []
    with open('{0}.tsv'.format(dataset_name)) as ifile:
        for line in ifile:
            tokens = line.strip().split('\t')
            data.append([float(tk) for tk in tokens[:-1]])
            labels.append(tokens[-1])
    data = np.array(data)
    labels = np.array(labels)
    return data, labels


Lets visualize the decision boundaries between these three classifications based on the classifier we just built.

In [41]:
COLOUR_FIGURE = True
from matplotlib.colors import ListedColormap
from sklearn.neighbors import KNeighborsClassifier

feature_names2 = [
    'area',
    'perimeter',
    'compactness',
    'length of kernel',
    'width of kernel',
    'asymmetry coefficien',
    'length of kernel groove',
]
features2, labels2 = load_dataset('seeds')
from sklearn.cross_validation import KFold
classifier2 = KNeighborsClassifier(n_neighbors=1)  # Defaults to 5 if not specified

kf = KFold(len(features2), n_folds=5, shuffle=True)
means = []

for training, testing in kf:
    classifier2.fit(features2[training], labels2[training])
    prediction = classifier2.predict(features2[testing])
    
    #fraction of correct decions per fold
    curmean = np.mean(prediction == labels2[testing])
    means.append(curmean)
print('Mean Accuracy:  {:.1%}'.format(np.mean(means)))

def plot_decision(features, labels, num_neighbors=1):
    '''Plots decision boundary for KNN Classifier
    
    Parameters
    -----------
    features:  ndarray
    labels:    sequence
    
    Returns
    -----------
    fig:       Matplotlib Figure
    ax :       Matplotlib Axis
    '''
    y0, y1 = features2[:, 2].min() * .9, features2[:, 2].max() * 1.1
    x0, x1 = features2[:, 0].min() * .9, features2[:, 0].max() * 1.1
    X = np.linspace(x0, x1, 1000)
    Y = np.linspace(y0, y1, 1000)
    X, Y = np.meshgrid(X, Y)
    model = KNeighborsClassifier(num_neighbors)
    model.fit(features2[:, (0,2)], labels2)
    C = model.predict(np.vstack([X.ravel(), Y.ravel()]).T).reshape(X.shape)
    if COLOUR_FIGURE:
        cmap = ListedColormap([(1., .7, .7), (.7, 1., .7), (.7, .7, 1.)])
    else:
        cmap = ListedColormap([(1., 1., 1.), (.2, .2, .2), (.6, .6, .6)])
    fig,ax = plt.subplots(figsize=(10,5))
    ax.set_xlim(x0, x1)
    ax.set_ylim(y0, y1)
    ax.set_xlabel(feature_names2[0])
    ax.set_ylabel(feature_names2[2])
    ax.pcolormesh(X, Y, C, cmap=cmap)
    if COLOUR_FIGURE:
        cmap = ListedColormap([(1., .0, .0), (.1, .6, .1), (.0, .0, 1.)])
        ax.scatter(features[:, 0], features[:, 2], c=labels, cmap=cmap)
    else:
        for lab, ma in zip(range(3), "Do^"):
            ax.plot(features2[labels2 == lab, 0], features2[
                     labels2 == lab, 2], ma, c=(1., 1., 1.), ms=6)
    return fig,ax

Mean Accuracy:  88.6%


In [42]:
names2 = sorted(set(labels2))
labels2 = np.array([names2.index(ell) for ell in labels2])

fig, ax = plot_decision(features2, labels2)
fig.savefig('figure3sklearn.png')

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score

classifier2 = Pipeline([('norm', StandardScaler()), ('knn', classifier2)])
crossed = cross_val_score(classifier2, features2, labels2)
print('Result with prescaling: {}'.format(crossed))

features2 -= features2.mean(0)
features2 /= features2.std(0)
fig, ax = plot_decision(features2, labels2)
fig.savefig('figure4sklearn.png')

fig, ax = plot_decision(features2, labels2, 11)
fig.savefig('figure5sklearn_w_11_neighbors.png')

Result with prescaling: [ 0.95833333  0.92753623  0.79710145]


<img src="figure3sklearn.png" />

Figure 3


### Preprocessing Data Normalization

The Pipeline constructor takes a list of pairs (str,clf). Each pair corresponds to a step in the pipeline: the first element is a string naming the step, while the second element is the object that performs the transformation.

We need to normalize all of the features to a common scale. There are many solutions to this problem; a simple one is to normalize to z-scores. The z-score of a value is how far away from the mean it is, in units of standard deviation. After normalization, every feature is in the same units (technically, every feature is now dimensionless; it has no units) and we can more confidently mix dimensions. 

<img src="figure4sklearn.png" />

Figure 4


Prescaling brought the accuracy up to 93%. We can further refine these results by looking at the 11 nearest neighbors instead of the closest 1

<img src="figure5sklearn_w_11_neighbors.png" />

Figure 5
